In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import datetime
import time
from sklearn import preprocessing
from sys import path
path.append(r'E:\data_mining\project\IJCAI-17-口碑\koubei')
import base
import predict

import matplotlib
myfont = matplotlib.font_manager.FontProperties(fname=r'C:/Windows/Fonts/msyh.ttf')
matplotlib.use('qt4agg')  
#指定默认字体  
matplotlib.rcParams['font.sans-serif'] = ['SimHei']   
matplotlib.rcParams['font.family']='sans-serif'  
#解决负号'-'显示为方块的问题  
matplotlib.rcParams['axes.unicode_minus'] = False

C:\Program Files\Anaconda3\lib\site-packages\matplotlib\__init__.py:1357: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
#读取user_pay_count
user_pay_counts = base.ReadUserPayCountCSV();
#读取开店时间数据
shop_open_dates = base.ReadShopOpenDateCSV();
###读取商家信息
shop_infos = base.ReadShopInfoCSV();
##读取城市名称对照表
city_names = base.ReadCityNameCSV();
#读取休假表
calenders = base.ReadCalendarCSV();

In [3]:
mean_3_3_3_prediction = pd.read_csv('../data/train/mean/mean_3_3_3_prediction.txt', sep='\t', index_col=0, encoding='UTF-8')

In [4]:
arima_prediction = pd.read_csv('../data/train/arima/arima_prediction.txt', sep='\t', index_col=0, encoding='UTF-8')
arima_prediction = predict.FixFinalPredict(arima_prediction)

In [5]:
start = pd.to_datetime('2016-10-18')
end = pd.to_datetime('2016-10-31')
mean_3_3_3_evaluations = {}
arima_evaluations = {}
for id in range(1, 2001):
    print(id)
    y_test = base.getTestCount(user_pay_counts, id, start, end)
    mean_3_3_3_evaluations[id] = base.evaluation(y_test, mean_3_3_3_prediction.loc[id].values)
    arima_evaluations[id] = base.evaluation(y_test, arima_prediction.loc[id].values)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [6]:
mean_evaluation_df = pd.DataFrame.from_dict(mean_3_3_3_evaluations, orient='index')
arima_evaluation_df = pd.DataFrame.from_dict(arima_evaluations, orient='index')

In [7]:
train_df = pd.concat([mean_evaluation_df, arima_evaluation_df], axis=1)

In [8]:
train_df.columns = ['mean', 'arima']

In [9]:
train_df['type'] = train_df[['mean', 'arima']].idxmin(axis=1)

In [10]:
def changeType(t):
    if t == 'mean':
        return 0
    elif t == 'arima':
        return 1

In [11]:
train_df['type'] = train_df['type'].apply(changeType)

In [12]:
def judgePredIntersectType(s):
    #不相交 且mean 大于arima
    if len(s[s >= 0]) == 14:
        return 1
    #不相交 且mean 小于arima
    elif len(s[s <= 0]) == 14:
        return 2
    #相交
    else:
        return 0

In [13]:
intersects = mean_3_3_3_prediction - arima_prediction
intersects = intersects.apply(judgePredIntersectType, axis=1)

In [14]:
train_df['pred_intersect'] = intersects

In [15]:
train_data = pd.read_csv('../data/train/fill_train_data.txt', sep='\t', index_col=0, encoding='UTF-8')
train_data.columns = np.array(range(35))
X_train_data = train_data[np.array(range(21))]

In [16]:
train_df['pre_week_mean'] = train_data[np.array(range(14, 21))].mean(axis=1).astype(np.int)

In [17]:
def judgeMeanIntersectType(s):
    #不相交 且mean 小于pred
    if len(s[s >= 0]) == 14:
        return 1
    #不相交 且mean 大于pred
    elif len(s[s <= 0]) == 14:
        return 2
    #相交
    else:
        return 0

In [18]:
intersects = arima_prediction.sub(train_df['pre_week_mean'].squeeze(), axis=0)
intersects = intersects.apply(judgeMeanIntersectType, axis=1)
train_df['mean_arima_intersect'] = intersects

In [19]:
intersects.value_counts()

0    1001
2     591
1     408
dtype: int64

In [20]:
intersects = mean_3_3_3_prediction.sub(train_df['pre_week_mean'].squeeze(), axis=0)
intersects = intersects.apply(judgeMeanIntersectType, axis=1)
train_df['mean_mean_intersect'] = intersects

In [21]:
 df = X_train_data.loc[3][np.array(range(7, 14))]

In [23]:
def appendExtrelist(l, count):
    if len(l) == 0:
        l.append(count)
    elif l[len(l) - 1] != count:
        l.append(count)

def rebuildMaxlist(maxes):
    if len(maxes) < 2:
        return maxes
    
    threshold = 0.6
    remove_indexes = []
    for i in range(1, len(maxes) - 1):
        if (maxes[i] < maxes[i - 1] * threshold) and (maxes[i] < maxes[i + 1] * threshold):
            remove_indexes.append(i)
    
    rebuild_maxes = []
    for i in range(len(maxes)):
        if i not in remove_indexes:
            rebuild_maxes.append(maxes[i])
    
    return rebuild_maxes

def rebuildMinlist(mines):
    if len(mines) < 2:
        return mines
    
    threshold = 1.6
    remove_indexes = []
    for i in range(1, len(mines) - 1):
        if (mines[i] > mines[i - 1] * threshold) and (mines[i] > mines[i + 1] * threshold):
            remove_indexes.append(i)
    
    rebuild_mins = []
    for i in range(len(mines)):
        if i not in remove_indexes:
            rebuild_mins.append(mines[i])
            
    return rebuild_mins

#前14天趋势
#def 0 持平 1 上升 2 下降
def getPeriodTrend(id, start_index, end_index, threshold, user_pay_counts, shop_open_dates, calenders):

    df = X_train_data.loc[id][np.array(range(start_index, end_index))]
    print(df)
    maxes = []
    mins = []
    current_trend = 0
    prev_trend = 0;
    for i in df.index:
        if i == start_index:
            continue
        
        if df[i] > df[i - 1]:
            current_trend = 1
        elif df[i] < df[i - 1]:
            current_trend = 2
        else:
            current_trend = 0
                
        if i == start_index + 1:
            if current_trend == 1:
                appendExtrelist(mins, df[i - 1])
            elif current_trend == 2:
                appendExtrelist(maxes, df[i - 1])
        elif i == end_index - 1:
            if current_trend != prev_trend:
                if prev_trend == 0:
                    if current_trend == 1:
                        appendExtrelist(mins, df[i - 1])
                    else:
                        appendExtrelist(maxes, df[i - 1])
                elif prev_trend == 1:
                    appendExtrelist(maxes, df[i - 1])
                else:
                    appendExtrelist(mins, df[i - 1])
                
            if current_trend == 1:
                appendExtrelist(maxes, df[i])
            elif current_trend == 2:
                appendExtrelist(mins, df[i])
            else:
                appendExtrelist(maxes, df[i])
                appendExtrelist(mins, df[i])
        elif current_trend != prev_trend:
            if prev_trend == 0:
                if current_trend == 1:
                    appendExtrelist(mins, df[i - 1])
                else:
                    appendExtrelist(maxes, df[i - 1])
            elif prev_trend == 1:
                appendExtrelist(maxes, df[i - 1])
            else:
                appendExtrelist(mins, df[i - 1])
        
        prev_trend = current_trend
    
    print(maxes)
    print(mins)
    maxes = rebuildMaxlist(maxes)
    mins = rebuildMinlist(mins)
    max_trend = 0
    min_trend = 0
    if len(maxes) == 1:
        maxes = maxes + maxes
    elif len(maxes) == 0:
        maxes = [0, 0] + maxes
    
    print(maxes)
    print(mins)
    if len(mins) == 1:
        mins = mins + mins
    elif len(mins) == 0:
        mins = [0, 0] + mins
    
    max_mean = np.mean(maxes)
    min_mean = np.mean(mins)
    if np.mean(np.diff(maxes)) <= -max_mean * threshold:
        max_trend = 2
    elif np.mean(np.diff(maxes)) >= max_mean * threshold:
        max_trend = 1
    else:
        max_trend = 0
        
    if np.mean(np.diff(mins)) <= -min_mean * threshold:
        min_trend = 2
    elif np.mean(np.diff(mins)) >= min_mean * threshold:
        min_trend = 1
    else:
        min_trend = 0
        
    if max_trend == 0:
        return min_trend
    elif max_trend == 1:
        if min_trend == 2:
            return 0
        else:
            return 1
    elif max_trend == 2:
        if min_trend == 1:
            return 0
        else:
            return 2

In [24]:
prev_trends = []
threshold = 0.2
for id in range(1, 2001):
    print(id)
    prev_trends.append(getPeriodTrend(id, 7, 21, threshold, user_pay_counts, shop_open_dates, calenders))
    #print(getPeriodTrend(id, 7, 21, threshold, user_pay_counts, shop_open_dates, calenders))

1
7     223
8     206
9     220
10    241
11    272
12    240
13    225
14    254
15    239
16    228
17    244
18    264
19    264
20    207
Name: 1, dtype: int64
[223, 272, 254, 264]
[206, 225, 228, 207]
[223, 272, 254, 264]
[206, 225, 228, 207]
2
7      77
8      82
9      71
10     71
11     96
12    121
13     67
14     34
15     34
16     34
17    112
18    161
19     95
20     34
Name: 2, dtype: int64
[82, 121, 161]
[77, 71, 34]
[82, 121, 161]
[77, 71, 34]
3
7      62
8      59
9      55
10     79
11    112
12     83
13     88
14     68
15     63
16     56
17     71
18    119
19     97
20     75
Name: 3, dtype: int64
[62, 112, 88, 119]
[55, 83, 56, 75]
[62, 112, 88, 119]
[55, 83, 56, 75]
4
7      74
8     101
9     102
10     98
11    203
12    199
13     91
14     86
15    113
16     71
17    121
18    165
19    200
20     89
Name: 4, dtype: int64
[102, 203, 113, 200]
[74, 98, 86, 71, 89]
[102, 203, 200]
[74, 98, 86, 71, 89]
5
7     277
8     272
9     309
10    182
11    179
1

In [25]:
#train_df['prev_trend'] = prev_trends

In [26]:
X_train_df = pd.DataFrame()
X_train_df['max'] = X_train_data.max(axis=1)
X_train_df['min'] = X_train_data.min(axis=1)
X_train_df['std'] = X_train_data.std(axis=1)
X_train_df['var'] = X_train_data.var(axis=1)
X_train_df['mean'] = X_train_data.mean(axis=1)
X_train_df['median'] = X_train_data.median(axis=1)

In [27]:
X_train_df['mean_max'] = mean_3_3_3_prediction.max(axis=1)
X_train_df['mean_min'] = mean_3_3_3_prediction.min(axis=1)
X_train_df['mean_std'] = mean_3_3_3_prediction.std(axis=1)
X_train_df['mean_var'] = mean_3_3_3_prediction.var(axis=1)
X_train_df['mean_mean'] = mean_3_3_3_prediction.mean(axis=1)
X_train_df['mean_median'] = mean_3_3_3_prediction.median(axis=1)

X_train_df['arima_max'] = arima_prediction.max(axis=1)
X_train_df['arima_min'] = arima_prediction.min(axis=1)
X_train_df['arima_std'] = arima_prediction.std(axis=1)
X_train_df['arima_var'] = arima_prediction.var(axis=1)
X_train_df['arima_mean'] = arima_prediction.mean(axis=1)
X_train_df['arima_median'] = arima_prediction.median(axis=1)

In [28]:
X_train_df['diff_arima_max'] = arima_prediction.max(axis=1) - X_train_data.max(axis=1)
X_train_df['diff_arima_min'] = arima_prediction.min(axis=1) - X_train_data.min(axis=1)
X_train_df['diff_arima_std'] = arima_prediction.std(axis=1) - X_train_data.std(axis=1)
X_train_df['diff_arima_var'] = arima_prediction.var(axis=1)- X_train_data.var(axis=1)
X_train_df['diff_arima_mean'] = arima_prediction.mean(axis=1) - X_train_data.mean(axis=1)
X_train_df['diff_arima_median'] = arima_prediction.median(axis=1) - X_train_data.median(axis=1)

X_train_df['diff_mean_max'] = mean_3_3_3_prediction.max(axis=1) - X_train_data.max(axis=1)
X_train_df['diff_mean_min'] = mean_3_3_3_prediction.min(axis=1) - X_train_data.min(axis=1)
X_train_df['diff_mean_std'] = mean_3_3_3_prediction.std(axis=1) - X_train_data.std(axis=1)
X_train_df['diff_mean_var'] = mean_3_3_3_prediction.var(axis=1) - X_train_data.var(axis=1)
X_train_df['diff_mean_mean'] = mean_3_3_3_prediction.mean(axis=1) - X_train_data.mean(axis=1)
X_train_df['diff_mean_median'] = mean_3_3_3_prediction.median(axis=1) - X_train_data.median(axis=1)


X_train_df['diff_mean_arima_max'] = mean_3_3_3_prediction.max(axis=1) - arima_prediction.max(axis=1)
X_train_df['diff_mean_arima_min'] = mean_3_3_3_prediction.min(axis=1) - arima_prediction.min(axis=1)
X_train_df['diff_mean_arima_std'] = mean_3_3_3_prediction.std(axis=1) - arima_prediction.std(axis=1)
X_train_df['diff_mean_arima_var'] = mean_3_3_3_prediction.var(axis=1) - arima_prediction.var(axis=1)
X_train_df['diff_mean_arima_mean'] = mean_3_3_3_prediction.mean(axis=1) - arima_prediction.mean(axis=1)
X_train_df['diff_mean_arima_median'] = mean_3_3_3_prediction.median(axis=1) - arima_prediction.median(axis=1)

In [29]:
X_train_scaled = preprocessing.scale(X_train_df.values)
X_train_df = pd.DataFrame(X_train_scaled)
X_train_df = X_train_df.set_index(np.array(range(1, 2001)))
X_train_df = pd.merge(X_train_df,  pd.get_dummies(train_df['pred_intersect'], prefix='pred_intersect').astype(np.int), left_index=True, right_index=True)
X_train_df = pd.merge(X_train_df,  pd.get_dummies(train_df['mean_arima_intersect'], prefix='mean_arima_intersect').astype(np.int), left_index=True, right_index=True)
X_train_df = pd.merge(X_train_df,  pd.get_dummies(train_df['mean_mean_intersect'], prefix='mean_mean_intersect').astype(np.int), left_index=True, right_index=True)

In [30]:
#X_train_df['prev_trend'] = train_df['prev_trend']

In [31]:
y_train_df = train_df['type']
y_train_df = y_train_df.to_frame()

In [32]:
today = datetime.date.today()

In [330]:
X_train_df.to_csv('../data/train/mean_arima/X_train' + today.isoformat() + '.csv')

In [331]:
y_train_df.to_csv('../data/train/mean_arima/y_train' + today.isoformat() + '.csv')

In [ ]:
train_df['prev_trend'].value_counts()

In [ ]:
def plotShopPayCounts(shop_id, date_range, predicts, names, evaluations, predict_num):
    counts = base.countShopPayTimePeriods(user_pay_counts, shop_id, date_range, [datetime.timedelta(hours=0), datetime.timedelta(hours=23)])
    #counts = base.WipeInvalidUserPayCount(counts)
    counts['holiday'] = calenders.loc[counts.index.strftime('%Y-%m-%d')]['daytype'].values
    info = shop_infos.ix[shop_id]
    weathers = base.readWeatherData(city_names.loc[info['city']])
    counts['weather'] = weathers.loc[counts.index]['desc'].values
    print(counts['count'].values)
    counts = counts.reset_index(drop=True)
    
    train_index = np.array(range(len(counts.index)))
    predict_index = np.array(range(len(counts.index)- predict_num, len(counts.index)))
    mean_index = np.array(range(len(counts.index)- 7 - predict_num, len(counts.index) - predict_num))
    f, (ax) = plt.subplots(1, 1, figsize=(10, 5))
    ax.plot(train_index, counts['count'])
    
    temp = counts[counts['holiday'] == 1]
    ax.scatter(temp.index, temp['count'], color='r')
    temp = counts[counts['holiday'] == 2]
    ax.scatter(temp.index, temp['count'], color='g')
    temp = counts[counts['holiday'] == 3]
    ax.scatter(temp.index, temp['count'], color='b')
    temp = counts[counts['holiday'] == 4]
    ax.scatter(temp.index, temp['count'], color='k')
    temp = counts[counts['holiday'] == 5]
    ax.scatter(temp.index, temp['count'], color='y')
    
    for i, name in enumerate(names):
        ax.plot(predict_index, predicts[i].loc[id], label=name)
        print("%s : %f"%(name, evaluations[i][id]))
    
#     ax.text(left, bottom, 'left top',
#         horizontalalignment='left',
#         verticalalignment='top',
#         transform=ax.transAxes)
    
    plt.title(info.cate_1_name + ' ' + info.cate_2_name + ('' if info.cate_3_name is None else info.cate_3_name), fontproperties=myfont)
    mean_value = np.int(train_data.loc[id][np.array(range(7, 21))].mean())
    print('pre_week_mean_value=' + str(mean_value))
    ax.axhline(y=mean_value, linewidth=1, color='r')
    plt.legend()
    plt.show()

In [ ]:
positive_train_df = train_df[train_df['type'] == 1]
negative_train_df = train_df[train_df['type'] == 0]

In [ ]:
# from mpl_toolkits.mplot3d import Axes3D

# fig = plt.figure()
# ax = fig.add_subplot(111,projection='3d')
# ax.bar(positive_train_df['type'], positive_train_df['mean_mean_intersect'], positive_train_df['pred_intersect'], zdir='y', color='b', alpha=0.8)
# ax.bar(negative_train_df['mean_arima_intersect'], negative_train_df['mean_mean_intersect'], negative_train_df['pred_intersect'], zdir='y', color='r', alpha=0.8)

# ax.set_xlabel('mean_arima')
# ax.set_ylabel('mean_mean')
# ax.set_zlabel('pred')

# plt.show()

In [ ]:
# f, (ax) = plt.subplots(1, 1, figsize=(20, 10))

# ax.scatter(negative_train_df['pred_intersect'], negative_train_df['mean_arima_intersect'] ,color='blue',s=8,edgecolor='none', alpha = 0.5, label='mean')
# ax.scatter(positive_train_df['pred_intersect'], positive_train_df['mean_arima_intersect'] ,color='red',s=8,edgecolor='none', alpha = 0.5, label='arima')

# ax.set_xlabel('diff_mean')
# ax.set_ylabel('diff_std')
# plt.legend()
# plt.show()

In [ ]:
combine_evaluation_df.mean()

In [ ]:
diff_evaluations = mean_evaluation_df - arima_evaluation_df

In [ ]:
sort_diff_diff_evaluations = diff_evaluations.sort_values(0)
sort_diff_diff_evaluations.reset_index(drop=True)

In [ ]:
evaluations = [mean_3_3_3_evaluations, arima_evaluations]

In [ ]:
predicts = [mean_3_3_3_prediction, arima_prediction]
names = ['3_3_3', 'arima']

In [248]:
#i=1900
406 1897 428 1136 1550 507 334 1762 10 1788 454 1799 1272 1529 794 1725 55 319 569 906 1688 938 954 603 316 1400 1333 769 97 1842	1
1185 158 1585 1789 1478 1323 964 465 1818 1701 1808 939 1554 501 1308 1904 1713 1800 473

In [ ]:
i  = 0

In [ ]:
#id = sort_diff_diff_evaluations.index[i]
#i = i + 1
id = 809
print(id)
#open_time = shop_open_dates.loc[id]['date'];
open_time = pd.to_datetime('2016-8-13')
start_time = open_time;
end_time = pd.to_datetime('2016-10-31')
print(open_time)
plotShopPayCounts(id, [start_time, end_time], predicts, names, evaluations, 14)

In [191]:
diff_evaluation = mean_evaluation_df - mean_arima_evaluation_df

NameError: name 'mean_arima_evaluation_df' is not defined

In [101]:
mean_evaluation_data_df = pd.concat([diff_mean, diff_evaluation], axis=1)

In [35]:
combine_evaluations = {}
for id in range(1, 2001):
    if abs(train_df['diff_mean'][id]) < 20 and abs(train_df['diff_std'][id]) < 10:
        combine_evaluations[id] = (train_df['arima'][id] + train_df['mean'][id]) / 2
    else:
        combine_evaluations[id] = train_df['mean'][id]
        
combine_evaluation_df = pd.DataFrame.from_dict(combine_evaluations, orient='index')

KeyError: 'diff_mean'

In [103]:
mean_evaluation_data_df.columns = ['mean', 'evaluation']

In [104]:
mean_evaluation_data_df.to_csv('../data/compare/mean_evaluation.csv')

In [136]:
filter_prediction_df = pd.DataFrame(filter_prediction)

In [137]:
filter_prediction_df = filter_prediction_df.T

In [ ]:
min_evaluations = 

In [156]:
# X_train_df['last_max'] = X_train_data.max(axis=1) == X_train_data[20]
# X_train_df['last_min'] = X_train_data.min(axis=1) == X_train_data[20]
# X_train_df[['last_max', 'last_min']] = X_train_df[['last_max', 'last_min']].applymap(lambda x : 1 if x == True else 0)

In [340]:
y_pred_df = pd.read_csv('../data/train/mean_arima/y_predict_gbdt.csv', index_col=0, encoding='UTF-8')

In [341]:
combine_evaluations = {}
sub_mean_evaluations = {}
for id in y_pred_df.index:
    if y_pred_df.loc[id].values == 0:
        combine_evaluations[id] = train_df['mean'][id];
    else:
        combine_evaluations[id] = train_df['arima'][id]
    
    sub_mean_evaluations[id] = train_df['mean'][id];
combine_evaluation_df = pd.DataFrame.from_dict(combine_evaluations, orient='index')
sub_mean_evaluation_df = pd.DataFrame.from_dict(sub_mean_evaluations, orient='index')

In [342]:
combine_evaluation_df.mean()

0    0.081468
dtype: float64

In [343]:
sub_mean_evaluation_df.mean()

0    0.083026
dtype: float64